## Wikipedia Article Retrieval Tool

This section defines a simple Python function, `get_article`, that uses the `wikipedia` Python package to search for and retrieve the content of a Wikipedia article based on a search term. This function will be used as a tool for the agent to fetch up-to-date information from Wikipedia.

In [ ]:
import wikipedia

def get_article(search_term):
    results = wikipedia.search(search_term)
    first_result = results[0]
    page = wikipedia.page(first_result, auto_suggest=False)
    return page.content

## Example: Fetching and Previewing Wikipedia Articles

Here, we demonstrate how to use the `get_article` function to retrieve and preview the content of Wikipedia articles for various search terms, such as "Avengers: Doomsday", "Nezha 2", "History of Malaysia", and "Iron Man". Only a preview of the article content is printed for brevity.

In [ ]:
article = get_article("Avengers: Doomsday")
print(article[:1000]) # article is very long, so let's just print a preview

In [ ]:
article = get_article("Nezha 2")
print(article[:500]) # article is very long, so let's just print a preview

In [ ]:
article = get_article("History of Malaysia")
print(article[:3000]) #article is super long so let's just print a preview

In [ ]:
article = get_article("Iron Man")
print(article[:1000]) #article is super long so let's just print a preview

## Tool Schema Definition for Agent Use

This cell defines a tool schema dictionary, `article_search_tool`, which describes the Wikipedia retrieval tool in a format compatible with agentic LLM APIs (like Anthropic Claude). The schema includes the tool's name, description, input schema, and required fields.

In [ ]:
article_search_tool = {
    "name": "get_article",
    "description": "A tool to retrieve an up to date Wikipedia article.",
    "input_schema": {
        "type": "object",
        "properties": {
            "search_term": {
                "type": "string",
                "description": "The search term to find a wikipedia article by title"
            },
        },
        "required": ["search_term"]
    }
}

## Setting Up Anthropic Client and Making a Tool-Use Request

This section initializes the Anthropic API client and demonstrates how to send a message to the Claude model, asking a question that may require tool use (e.g., "What is the box office for Nezha 2?"). The tool schema is provided to the model, enabling it to call the Wikipedia tool if needed.

In [ ]:
from anthropic import Anthropic
from dotenv import load_dotenv

load_dotenv()

client = Anthropic()

messages = [{"role": "user", "content": "What is the box office for Nezha 2"}]

response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )

In [ ]:
response.content

## Appending the Assistant's Tool Use to the Conversation

This cell appends the assistant's tool use response to the ongoing conversation history, preparing for the next step in the agentic interaction.

In [ ]:
messages.append({"role": "assistant", "content": response.content})

## Extracting Tool Use Information

This section extracts the tool name and input parameters from the model's tool use response, so that the tool can be called programmatically.

In [ ]:
# This is a simple, but brittle way of getting the tool use information
# We're simply taking the last block from Claude's response.
tool_use = response.content[-1]
tool_name = tool_use.name
tool_input = tool_use.input
print("Tool name: ", tool_name)
print("Tool input", tool_input)

## Executing the Tool and Returning Results

Here, the code checks if the model requested the Wikipedia tool, executes the tool with the provided input, and prints a preview of the Wikipedia article content.

In [ ]:
if tool_name == "get_article":
    search_term = tool_input["search_term"]
    wiki_result = get_article(search_term)
    print(f"Searching Wikipedia for {search_term}")
    print("WIKIPEDIA PAGE CONTENT:")
    print(wiki_result[:500]) #just printing a small bit of the article because it's so long

## Constructing a Tool Result Message

This cell shows how to construct a message containing the tool result, formatted according to the agentic API's expectations, so it can be sent back to the model.

In [ ]:
{
  "role": "user",
  "content": [
    {
      "type": "tool_result",
      "tool_use_id": "toolu_01A09q90qw90lq917835lq9",
      "content": "The result of actually calling the tool goes here"
    }
  ]
}

## Sending the Tool Result to the Conversation

The tool result message is appended to the conversation history, allowing the agent to use the tool's output in its next response.

In [ ]:
tool_response = {
    "role": "user",
    "content": [
        {
        "type": "tool_result",
        "tool_use_id": tool_use.id,
        "content": wiki_result
        }
    ]
}

In [ ]:
tool_response

In [ ]:
messages.append(tool_response)

In [ ]:
follow_up_response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    messages=messages,
    max_tokens=1000,
    tools=[article_search_tool]
)

In [ ]:
follow_up_response.content[0].text

## Model Follow-up: Final Answer Generation

This section sends the updated conversation (including the tool result) back to the model, prompting it to generate a final answer that incorporates the information retrieved from Wikipedia.

In [ ]:

def answer_question(question):
    messages = [{"role": "user", "content": question}]

    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        messages=messages,
        max_tokens=1000,
        tools=[article_search_tool]
    )
    
    if(response.stop_reason == "tool_use"):
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
        #Add Claude's tool use call to messages:
        messages.append({"role": "assistant", "content": response.content})

        if tool_name == "get_article":
            search_term = tool_input["search_term"]
            print(f"Claude wants to get an article for {search_term}")
            wiki_result = get_article(search_term) #get wikipedia article content
            #construct our tool_result message
            tool_response = {
                "role": "user",
                "content": [
                    {
                    "type": "tool_result",
                    "tool_use_id": tool_use.id,
                    "content": wiki_result
                    }
                ]
                }
            messages.append(tool_response)
            #respond back to Claude
            response = client.messages.create(
                model="claude-3-7-sonnet-20250219",
                messages=messages,
                max_tokens=1000,
                tools=[article_search_tool]
            )
            print("Claude's final answer:")
            print(response.content[0].text)

    else:
        print("Claude did not call our tool")
        print(response.content[0].text)

## Agentic Tool Use: Full Question-Answering Loop

This cell defines a reusable function, `answer_question`, that demonstrates the full agentic tool-use loop: sending a question to the model, detecting tool use, executing the tool, sending the result, and printing the model's final answer.

In [ ]:
answer_question("What are the names of all the Avengers films in the Marvel Cinematic Universe?")

## Web Search with Anthropic Claude API

This code demonstrates how to use Anthropic's Claude API to perform web searches using the built-in web_search_20250305 tool. This allows Claude to search the web for current information and provide up-to-date responses.

### Overview
The code creates a simple web search query using Claude's web search capability,which is particularly useful for getting current information that might not be available in the model's training data.

### Key Components

### Imports
- `anthropic`: The official Anthropic Python client
- `IPython.display`: For enhanced display formatting in Jupyter notebooks
- `json`: For pretty-printing response objects

### API Configuration
- Model: `claude-opus-4-1-20250805` (latest Claude Opus model)
- Max tokens: 1024 (controls response length)
- Tool: `web_search_20250305` with max 5 uses per request

### Usage

1. **Setup**: Ensure you have the Anthropic API key configured
2. **Query**: Modify the user message to ask any question requiring current information
3. **Execute**: Run the code to get Claude's response with web search results

### Example Use Cases
* Getting current software version information
* Finding recent news or updates
* Researching current events
* Checking latest technology trends
* Verifying current facts or statistics

### Response Format
The response will contain:
* Claude's analysis of the search results
* Relevant information found through web search
* Citations to source URLs when available

### Cost Considerations
* Web search tool usage incurs additional costs
* Each search counts toward your API usage quota
* Monitor usage in your Anthropic dashboard


In [ ]:
import anthropic
from IPython.display import display, Markdown
import json 

client = anthropic.Anthropic()

response = client.messages.create(
    model="claude-opus-4-1-20250805",
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Can you teach me how to code an AI agent using the Anthropic API?"
        }
    ],
    tools=[{
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 5
    }]
)
print(response)

In [ ]:
# Option 1: Pretty print the entire response as JSON
print("=== Full Response ===")
print(json.dumps(response.model_dump(), indent=2))

# Option 2: Display just the content in a formatted way
print("\n=== Response Content ===")
for i, content_block in enumerate(response.content):
    print(f"\n--- Content Block {i+1} ---")
    print(f"Type: {content_block.type}")
    if hasattr(content_block, 'text'):
        print(f"Text: {content_block.text}")
    if hasattr(content_block, 'name'):
        print(f"Tool Name: {content_block.name}")
        print(f"Tool Input: {content_block.input}")

# Option 3: Display as markdown for better formatting
print("\n=== Formatted Display ===")
formatted_output = "## Claude Response\n\n"
for i, content_block in enumerate(response.content):
    formatted_output += f"### Block {i+1}\n"
    formatted_output += f"**Type:** {content_block.type}\n\n"
    
    if hasattr(content_block, 'text'):
        formatted_output += f"**Content:**\n{content_block.text}\n\n"
    if hasattr(content_block, 'name'):
        formatted_output += f"**Tool:** {content_block.name}\n"
        formatted_output += f"**Input:** {content_block.input}\n\n"

display(Markdown(formatted_output))